## Kymograph Cluster Code

In [1]:
import trenchripper as tr

#### Start Dask

In [2]:
kymo_controller = tr.cluster.dask_controller(walltime='04:00:00',local=False,n_workers=40,memory='2GB')
kymo_controller.startdask()
kymo_controller.daskcluster.start_workers()

In [15]:
kymo_controller.daskclient

<Client: scheduler='tcp://10.120.16.160:42949' processes=10 cores=10>

#### Perform Extraction

In [16]:
hdf5_extractor = tr.ndextract.hdf5_fov_extractor("/n/scratch2/de64/for_sylvia/Bacillus_revival_12_7_2020.nd2","/n/scratch2/de64/full_pipeline_test/hdf5")

In [17]:
kymo_controller.mapfovs(hdf5_extractor.extract_fov,40)

In [18]:
props = kymo_controller.daskclient.gather(kymo_controller.futures) #this will hang until all futures are done

distributed.utils - ERROR - 'start'
Traceback (most recent call last):
  File "/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/distributed/utils.py", line 646, in log_errors
    yield
  File "/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/distributed/bokeh/components.py", line 521, in update
    ts = metadata['keys'][self.key]
KeyError: 'start'
tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x7f93242dbd08>, <Future finished exception=KeyError('start',)>)
Traceback (most recent call last):
  File "/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/tornado/ioloop.py", line 758, in _run_callback
    ret = callback()
  File "/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/tornado/stack_context.py", line 300, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/tornado/ioloop.py", li

#### Generate Kymographs

In [3]:
input_file_prefix = '/n/groups/paulsson/Daniel/Image_analysis_pipeline/tiff_extraction/test_out_4/fov_'
output_path = '/n/groups/paulsson/Daniel/Image_analysis_pipeline/kymographs_6'
all_channels = ["channel_BF","channel_RFP"]

kymo = tr.kymograph.kychunker(input_file_prefix=input_file_prefix,output_path=output_path,all_channels=all_channels,trench_len_y=270,padding_y=20,trench_width_x=30,\
                 t_chunk=25,y_percentile=85,y_min_edge_dist=50,smoothing_kernel_y=(9,1),triangle_nbins=50,triangle_scaling=1.,\
                 top_orientation=0,x_percentile=85,background_kernel_x=(301,1),smoothing_kernel_x=(9,1),otsu_nbins=50,otsu_scaling=1.)

In [6]:
kymo_controller.mapfovs(kymo.generate_kymograph,30)

In [7]:
props = kymo_controller.daskclient.gather(kymo_controller.futures) #this will hang until all futures are done

#### End Dask, Cancel Jobs

In [19]:
kymo_controller.shutdown()